# A 5-year validated Nitrate Dataset from the Pioneer-NES array

The Ocean Observatories Initiative (OOI) deployed both the In-Situ Ultraviolet Spectrophotometer (ISUS) and Submersible Underwater Nitrate Sensor (SUNA) for continuous, in-situ measurement of nitrate. At the Pioneer-New England Shelf Array (Pioneer-NES), ISUS/SUNA sensors were deployed at 7-meters depth at the Inshore (ISSM), Central (CNSM), and Offshore (OSSM) Surface Mooring locations. The SUNA sensor replaced the ISUS sensors spring 2018. The SUNA was a major improvement in technology, with significant improvements in accuracy and precision. However, it still suffers from calibration drift due to lamp fatigue and biofouling as well as spectral interference due to bromide and fluorometric CDOM. 

This notebook implements the recommendations from the _OOI Biogeochemical Sensor Data: Best Practices and User Guide_ for correcting and validating the SUNA Data. Drift is corrected by application of post-cruise calibrations to recalculate the temperature-and-salinity corrected nitrate concentration following Sakamoto (2009a) and estimating a linear drift between pre-and-post cruise deployments. Validation is performed by comparison with discrete water samples collected during deployment/recovery of the sensors, with correction for offsets and drift due to biofouling.

#### Load Libraries

This notebook makes extensive use of  OOI-developed python libraries: ```ooi_data_explorations```. This is availble on gitHub with instructions for setup.

In [1]:
import os, re, sys, ast
import numpy as np
import pandas as pd
import xarray as xr

# Load ooi_data_explorations toolset
from ooi_data_explorations.common import get_annotations, get_vocabulary, load_kdata, add_annotation_qc_flags,  get_deployment_dates, list_deployments, get_sensor_information
from ooi_data_explorations.combine_data import combine_datasets
from ooi_data_explorations.uncabled.process_nutnr import suna_datalogger, suna_instrument, drift_correction
from ooi_data_explorations.bottles import clean_data

# Import the utils/functions
from utils import *

# Matplotlib for plotting functions
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
check = xr.open_dataset('../data/Pioneer-NES/ISSM/CP03ISSM-RID26-07-NUTNRB000_final.nc')
check

<xarray.Dataset>
Dimensions:                                  (time: 154707)
Coordinates:
  * time                                     (time) datetime64[ns] 2018-03-25...
Data variables: (12/13)
    serial_number                            (time) int64 ...
    deployment                               (time) int64 ...
    sea_water_practical_salinity             (time) float64 ...
    sea_water_temperature                    (time) float64 ...
    nitrate_concentration                    (time) float64 ...
    nitrate_concentration_qc_flag            (time) int64 ...
    ...                                       ...
    corrected_nitrate_concentration_qc_flag  (time) int32 ...
    drift_corrected_nitrate                  (time) float64 ...
    drift_corrected_nitrate_qc_flag          (time) int64 ...
    bottle_corrected_nitrate                 (time) float64 ...
    bottle_corrected_nitrate_qc_flag         (time) int32 ...
    burst_median_absolute_deviation          (time) float64 ...
Attributes: (12/69)
    node:                               RID26
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             suna_dcl_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.36718
    lon:                                -70.88215

In [19]:
check.time.min(), check.time.max()

(<xarray.DataArray 'time' ()>
 array('2018-03-25T15:30:00.000000000', dtype='datetime64[ns]'),
 <xarray.DataArray 'time' ()>
 array('2022-11-13T11:45:00.000000000', dtype='datetime64[ns]'))

In [8]:
check.time.min(), check.time.max()

(<xarray.DataArray 'time' ()>
 array('2018-03-24T14:45:00.000000000', dtype='datetime64[ns]'),
 <xarray.DataArray 'time' ()>
 array('2022-10-23T17:00:00.000000000', dtype='datetime64[ns]'))

---
## Datasets

SUNA datasets are known as “NUTNR” on OOI (Nutrient Sensor). NUTNR datasets predating spring 2018 are from the ISUS instrument and not used, due to known measurement issues that make a quantitative data quality assessment difficult. SUNA datasets at the Pioneer-NES array begin with deployment 9 for CNSM and deployment 8 for both ISSM and OSSM. SUNA datasets from the three moorings cover from March 2018 through November 2022, when the Pioneer-NES array was retired.

The SUNA datasets are downloaded on a deployment-by-deployment basis. Each dataset additionally contains the practical salinity and seawater temperature measured by the collocated CTD on the mooring NSIF. This data is interpolated to the SUNA dataset timestamps and integrated into the dataset by OOI. Salinity and temperature are necessary to calculate the Sakamoto (2009) nitrate correction for salinity interference.

In [ ]:
nes_datasets = pd.read_csv("../data/nes_datasets.csv")
nes_datasets['deployments'] = nes_datasets['deployments'].apply(ast.literal_eval)
nes_datasets

### Load the SUNA Data

The following steps are taken in order to download, and 
* Iterate through each individual deployment
* For each deployment method (telemetered/recovered_host/recovered_inst) :
    1. Load the dataset
    2. Process the datasets
    3. Run QC checks
    4. Burst average using median averaging to 15-minute intervals
* The average datasets are combined into a single "merged" dataset
* Save the results to local data directory as "_merged"

The following table outlines the relevant parameters in the dataset for the QC-checks that are performed:

| Parameter	| Suspect Threshold |	Fail Threshold |
| --------- | ----------------- | -------------- |
| RMSE of spectral fit	| >0.001	| >0.1| 
| Absorbance at 254 nm	| N/A	| >1.3| 
| Absorbance at 350 nm	| N/A	| >1.3| 
| Dark values	| N/A	| <0| 
| Spectrum average	| N/A	| <10000| 
| Nitrate concentration	| N/A	| <-2 or >3000| 


In [ ]:
# Set the reference designator
refdes = 'CP04OSSM-RID26-07-NUTNRB000'
site, node, sensor = refdes.split("-",2)

# Get the deployments
idx = nes_datasets[nes_datasets["array"] == site].index
deployments = nes_datasets.loc[idx[0]]['deployments']
if 'CNSM' in site:
    deployments = [x for x in deployments if x > 8]
else:
    deployments = [x for x in deployments if x >= 8]

In [ ]:
# Go deployment-by-deployment for the given reference designator, download the data from each delivery method
# add annotations, process, burst-average the data, combine the methods, and save
for dN in deployments:

    # Get a single deployment dataset
    dN = str(dN).zfill(4)

    # Grab the annotations to later add to the datasets
    annotations = get_annotations(site, node, sensor)

    # ----------------------------------------------------
    # Load and process the Telemetered data
    tdata = load_kdata(site, node, sensor, 'telemetered', 'suna_dcl_recovered', tag=f'deployment{dN}_{refdes}*.nc')
    # Add in annotation qc flags to the "unprocessed" dataset
    tdata = add_annotation_qc_flags(tdata, annotations)
    # Now process the data
    tdata = suna_datalogger(tdata, burst=False)
    # Resample the data
    tdata = burst_resample(tdata)

    # ----------------------------------------------------
    # Load and process the Recovered host data
    hdata = load_kdata(site, node, sensor, 'recovered_host', 'suna_dcl_recovered', tag=f'deployment{dN}_{refdes}*.nc')
    # Add in annotation qc flags to the "unprocessed" dataset
    hdata = add_annotation_qc_flags(hdata, annotations)
    # Now process the data
    hdata = suna_datalogger(hdata, burst=False)    # Resample the data
    # Resample the data
    hdata = burst_resample(hdata)

    # ----------------------------------------------------
    # Load and process the Recovered instrument data
    idata = load_kdata(site, node, sensor, 'recovered_inst', 'suna_instrument_recovered', tag=f'deployment{dN}_{refdes}*.nc')
    # Add in annotation qc flags to the "unprocessed" dataset
    idata = add_annotation_qc_flags(idata, annotations)
    # Now process the data
    idata = suna_instrument(idata, burst=False)
    # Resample the data
    idata = burst_resample(idata)

    # ----------------------------------------------------
    # Combine the data
    data = combine_datasets(tdata, hdata, idata, None)

    # Remove the data that can't be serialized in a netCDF
    del data['internal_timestamp'].attrs['calendar']
    del data['internal_timestamp'].attrs['units']

    # Save the merged dataset
    array = site[4:]
    data.to_netcdf(f"../data/Pioneer-NES/{array}/{refdes}_deployment{dN}_merged.nc")

In [ ]:
from ooi_data_explorations.common import get_deployment_dates, list_deployments

### Apply the Drift Correction
Drift in the observed nitrate is known to occur due to lamp aging and biofouling. The impact of lamp aging can be corrected for using a post-cruise calibration and calculating a linear change between the pre-and-post deployment calibrations following Palevsky et al (2023):

$$
\frac{\Delta NO_{3}}{day} = \frac{NO_{3}^{predeployment \ cal} - NO_{3}^{postdeployment \ cal}}{Predeployment \ date - postdeployment \ date}
$$

In practice, we don’t have the DI water dataset for the pre-and-post deployment calibrations, just the calibration files themselves. Instead, we recalculate the nitrate concentration timeseries using the post-deployment calibration and assume that the difference between the timeseries calculated with the pre-deployment calibration and post-deployment calibration at the start of the deployment is $\Delta NO_{3}$. This assumption is reasonable because, apart from burn-in, the sensor is lightly used pre-deployment and drift due to lamp aging is a function of lamp-use.

The procedure here is to:
* Load each _merged dataset
* Apply the drift correction
* Save the corrected dataset as _drift_corrected

In [ ]:
basepath = "/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES"
for mooring in os.listdir(basepath):
    if not mooring.endswith('SM'):
        continue
    else:
        mooringpath = "/".join((basepath, mooring))
        for dataset in sorted(os.listdir(mooringpath)):
            if dataset.endswith('_merged.nc'):
                filepath = "/".join((mooringpath, dataset))
                # Load the file
                refdes = dataset.split("_")[0]
                subsite, node, sensor = refdes.split("-", 2)
                data = xr.open_dataset(filepath)
                # Apply the drift correction
                data = drift_correction(data, subsite, node, sensor)
                # Save the results
                dataset = dataset.replace("_merged","_drift_corrected")
                new_filepath = "/".join((mooringpath, dataset))
                data.to_netcdf(new_filepath, format="netcdf4", engine='h5netcdf') 

### Merge Drift Corrected Datasets 
Now, we will merge all of the datasets which have had the drift correction applied for a given sensor and merge them into a single dataset

In [ ]:
basepath = "/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES"
for mooring in os.listdir(basepath):
    data = None
    if not mooring.endswith('SM'):
        continue
    else:
        mooringpath = "/".join((basepath, mooring))
        for file in sorted(os.listdir(mooringpath)):
            if "drift" in file:
                ds = xr.open_dataset(mooringpath + "/" + file)
                if data is None:
                    data = ds
                else:
                    data = xr.concat([data, ds], dim="time")
    # Apply the quality checks on the drift-corrected nitrate
    qc_flags = quality_checks(data, 'drift_corrected_nitrate')
    data["drift_corrected_nitrate_qc_flag"] = qc_flags
    data["drift_corrected_nitrate_qc_flag"].attrs = {
        'flag_values': np.array([1, 2, 3, 4, 9]),
        'flag_meanings': 'pass not_evaluated suspect_or_of_high_interest fail missing_data',
        'standard_name': 'drift_corrected_nitrate status_flag',
        'long_name': 'Nitrate Concentration - Temp, Sal, and Drift Corrected Quality Flag',
        'comment': ('This quality flag represents an assessment of the nitrate concentration '
                    'that is corrected for temperature, salinity following Sakamoto (2009), '
                    'and for instrument drift. Checks include assessment of RMSE of the spectral '
                    'measurements, absorptions at 254 nm and 350 nm wavelengths, dark values, '
                    'spectral averages, and a range check based on instrument calibration.')
    }
    
    # Update the drift corrected nitrate description
    data['drift_corrected_nitrate'].attrs['comment'] = ('The measured nitrate concentration that is corrected for temperature '
                'and salinity following Sakamoto (2009), with linear drift, estimated from the difference between pre-and-post cruise DI-water calibrations, removed.')

    # Need to update the qc_flags on the T-S corrected nitrate concentration
    qc_flags = quality_checks(data, 'corrected_nitrate_concentration')
    data["corrected_nitrate_concentration_qc_flag"] = qc_flags
    data["corrected_nitrate_concentration_qc_flag"].attrs = {
        'flag_values': np.array([1, 2, 3, 4, 9]),
        'flag_meanings': 'pass not_evaluated suspect_or_of_high_interest fail missing_data',
        'standard_name': 'bottle_corrected_nitrate status_flag',
        'long_name': 'Nitrate Concentration - Temp, Sal, and Drift Corrected Quality Flag',
        'comment': ('This quality flag represents an assessment of the nitrate concentration '
                    'that is corrected for temperature, salinity following Sakamoto (2009). '
                    'Checks include assessment of RMSE of the spectral '
                    'measurements, absorptions at 254 nm and 350 nm wavelengths, dark values, '
                    'spectral averages, and a range check based on instrument calibration.')
    }
    
    # Save the result
    filename = filepath.split("/")[-1].split("_")[0]
    savepath = mooringpath + "/" + filename + ".nc"
    data.to_netcdf(savepath, format="netcdf4", engine="h5netcdf")

### Load the processed and merged data

In [ ]:
cnsm = xr.open_dataset("/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/CNSM/CP01CNSM-RID26-07-NUTNRB000.nc")
ossm = xr.open_dataset("/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/OSSM/CP01CNSM-RID26-07-NUTNRB000.nc")
issm = xr.open_dataset("/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/ISSM/CP01CNSM-RID26-07-NUTNRB000.nc")

---
## Bottle Correction
One of the primary purposes of the discrete water sampling data is for the evaluation and validation of OOI instrumentation.  Following the methods described in Palevsky et al. (2023), we use the bottle data collected during deployment and recovery of the SUNA sensors to correct for offsets and drift. This is done with the following equation:

$$
\frac{\Delta NO_{3}}{day}=\frac{(NO_{3}^{bottle}-NO_{3}^{SUNA})_{deployment}-(NO_{3}^{bottle}-NO_{3}^{SUNA})_{recovery}}{Recovery\ date-deployment\ date}
$$

and

$$
\Delta NO_{3}^{offset}=(NO_{3}^{bottle}-NO_{3}^{SUNA})_{deployment}
$$

which yields the following equation for calculating the bottle-adjusted nitrate:

$$
NO_{3}^{adj}(t)=NO_{3}^{obs}(t)+\frac{\Delta NO_{3}}{day}*\Delta t+\Delta NO_{3}^{offset}
$$

### Load and Clean the Bottle Data
Next, we want to load and clean the bottle data for the Pioneer-NES array.

In [ ]:
from ooi_data_explorations.bottles import clean_data

In [ ]:
basepath = '/home/jovyan/ooi/cruise_data/pioneer-nes'
bottle_data = None

for cruise in sorted(os.listdir(basepath)):
    # Find the bottle data
    cruise_path = "/".join((basepath, cruise, "Water_Sampling"))
    if os.path.exists(cruise_path):
        discrete_file = [x for x in os.listdir(cruise_path) if x.endswith('Discrete_Summary.csv')]
        if len(discrete_file) == 0:
            continue
        else:
            discrete_file = discrete_file[0]
    else:
        continue
    
    # Load the individual cruise discrete file
    cruise_data = pd.read_csv("/".join((cruise_path, discrete_file)), index_col=None)

    # Merge into a single dataset
    if bottle_data is None:
        bottle_data = cruise_data
    else:
        bottle_data = pd.concat([bottle_data, cruise_data], ignore_index=True)
    
    

In [ ]:
bottle_data = clean_data(bottle_data)
bottle_data

In [ ]:
# For now, the Data on the Raw Data Repo isn't sufficient to cover all of the time period, so will use other sheets I prepared independently
cnsm_nitrate = pd.read_excel("../data/Pioneer-NES/Ship/CNSM_nitrate.xlsx", index_col=0)
issm_nitrate = pd.read_excel("../data/Pioneer-NES/Ship/ISSM_nitrate.xlsx", index_col=0)
ossm_nitrate = pd.read_excel("../data/Pioneer-NES/Ship/OSSM_nitrate.xlsx", index_col=0)

In [ ]:
# Match deployment and recovery numbers for each buoy with the datasets
def remove_last_letter(x):
    if x.endswith(('A','B')):
        x = x[0:-1]
    else:
        pass
    return x

In [ ]:
cnsm_nitrate['Cruise'] = cnsm_nitrate['Cruise'].apply(lambda x: remove_last_letter(x))
issm_nitrate['Cruise'] = issm_nitrate['Cruise'].apply(lambda x: remove_last_letter(x))
ossm_nitrate['Cruise'] = ossm_nitrate['Cruise'].apply(lambda x: remove_last_letter(x))

### Apply Bottle Offsets
This next section applies the bottle offsets to the nitrate data at the start of the deployment

First, the timeseries is smoothed using a 6-hour centered rolling average to reduce noise while preserving daily nutrient cycling patterns. Then, the deployment and recovery bottle nitrate and smoothed SUNA nitrate are matched based on time. If there is no SUNA nitrate data available within 3 days of the bottle data at recovery, we do not calculate a drift based on the bottle data. Instead, we apply only the offset correction using the $\Delta NO_{3}^{offset}$ value. Otherwise, we go ahead and calculate $\frac{\Delta NO_{3}}{day}$ and apply Eqn. 4.4 to arrive at our bottle-corrected nitrate $NO_{3}^{adj}$. 

In [ ]:
from ooi_data_explorations.common import get_sensor_information

In [ ]:
def get_deployment_info(site, node, sensor, deployments):
    keys = ['deploymentNumber', 'uid', 'deployStart', 'deployEnd', 'deployCruise', 'recoverCruise']
    deployInfo = {x: [] for x in keys}
    for dN in deployments:
        # Get the sensor info
        sensorInfo = get_sensor_information(site, node, sensor, dN)
    
        # With the sensor info for a given deployment, get relevant data
        assetUid = sensorInfo[0]['sensor']['uid']
    
        # Get deployment info
        deployCruise = sensorInfo[0]['deployCruiseInfo']['uniqueCruiseIdentifier']
        deployStart = sensorInfo[0]['eventStartTime']
        deployStart = pd.to_datetime(convert_time(deployStart))
    
        # Get recovery info
        recoverCruise = sensorInfo[0]['recoverCruiseInfo']['uniqueCruiseIdentifier']
        deployEnd = sensorInfo[0]['eventStopTime']
        deployEnd = pd.to_datetime(convert_time(deployEnd))
    
        # Save results
        deployInfo['deploymentNumber'].append(int(dN))
        deployInfo['uid'].append(assetUid)
        deployInfo['deployStart'].append(deployStart)
        deployInfo['deployEnd'].append(deployEnd)
        deployInfo['deployCruise'].append(deployCruise)
        deployInfo['recoverCruise'].append(recoverCruise)

    return deployInfo

In [ ]:
# Okay, it is easiest to do the operation on a deployment by deployment basis
def bottle_correction(ds, deployments, bottle_data):
    """Apply the bottle correction to a dataset"""
    # First, check that the deployments dataset index is set to the deploymentNumber
    if not deployments.index.name == 'deploymentNumber':
        deployments.set_index(keys='deploymentNumber', drop=True, inplace=True)

    # Next, get the unique deployment number of the dataset
    deployNum = np.unique(ds['deployment'])

    # Get the deployment and recovery cruises
    deployCruise = deployments.loc[deployNum]['deployCruise'].values[0]
    recoverCruise = deployments.loc[deployNum]['recoverCruise'].values[0]
    
    # Select the associated nitrate data
    deployBottles = bottle_data[bottle_data['Cruise'] == deployCruise].drop(columns='Cruise').groupby('Start Time [UTC]').mean()
    recoverBottles = bottle_data[bottle_data['Cruise'] == recoverCruise].drop(columns='Cruise').groupby('Start Time [UTC]').mean()

    # Next, filter the NUTNR data using a 6H rolling window
    smoothed_data = ds['drift_corrected_nitrate'].to_dataframe().rolling('6H', center=True, closed='both').mean()
    smoothed_data = xr.Dataset(smoothed_data)
    
    # Find the closest data point in the smoothed data to get the 
    deploy_NO3 = deployBottles.reset_index().mean()
    suna_NO3 = smoothed_data.sel(time=deploy_NO3['Start Time [UTC]'], method='nearest')
    
    # Calculate the bottle offset
    bottle_offset = deploy_NO3['Discrete Nitrate [uM]'] - suna_NO3['drift_corrected_nitrate'].data

    # Now add the offset to the smoothed suna data and full suna drift-corrected
    smoothed_data = smoothed_data + bottle_offset
    
    # With the data adjusted for the starting offset, calculate the difference at the end
    recover_NO3 = recoverBottles.reset_index().mean()
    suna_NO3 = smoothed_data.sel(time=recover_NO3['Start Time [UTC]'], method='nearest')

    # Check if the time difference between the bottle sample and the identified nearest 
    # SUNA measurement exceeds 3 days, in which case ONLY apply the initial offset
    if len(recoverBottles) == 0:
        delta_NO3 = xr.DataArray(
            data = np.zeros(np.shape(smoothed_data['time'])),
            dims = 'time',
            coords = dict(
                time=smoothed_data.time)) 
    elif np.abs(suna_NO3['time'].values - recover_NO3['Start Time [UTC]']).to_timedelta64() > pd.Timedelta('3 days'):
        delta_NO3 = xr.DataArray(
            data = np.zeros(np.shape(smoothed_data['time'])),
            dims = 'time',
            coords = dict(
                time=smoothed_data.time))   
    else:
        # Calculate the bottle-derived drift
        dNO3 = recover_NO3['Discrete Nitrate [uM]'] - suna_NO3['drift_corrected_nitrate'].data
        dt = recover_NO3['Start Time [UTC]'] - deploy_NO3['Start Time [UTC]']
        dNO3_dt = dNO3/dt.to_timedelta64().astype('int')
        delta_NO3 = (smoothed_data.time - np.datetime64(deploy_NO3['Start Time [UTC]'])).astype('int')*dNO3_dt

    # Add in the bottle correction to both the smoothed data and the drift-corrected-data
    smoothed_data = smoothed_data + delta_NO3
    smoothed_data['deployment'] = ds['deployment']
    ds['bottle_corrected_nitrate'] = ds['drift_corrected_nitrate'] + bottle_offset + delta_NO3
    
    return ds, smoothed_data, deployBottles, recoverBottles

##### CP01CNSM-RID26-07-NUTNRB

In [ ]:
# Get the needed deployment information
site, node, sensor = 'CP01CNSM-RID26-07-NUTNRB000'.split('-',2)
deployInfo = get_deployment_info(site, node, sensor, deployments)
deployInfo = pd.DataFrame(deployInfo)
deployInfo.set_index(keys='deploymentNumber', drop=True, inplace=True)
deployInfo

In [ ]:
# Apply the bottle corrections to CNSM data
new_cnsm = None
smoothed_cnsm = None
deploy_bottles = None
recover_bottles = None
for depNum in np.unique(cnsm['deployment']):
    depdata = cnsm.where(cnsm.deployment == depNum, drop=True)
    depdata, smoothed_data, deployBottles, recoverBottles = bottle_correction(depdata, deployInfo, cnsm_nitrate)
    deployBottles['Deployment'] = depNum
    recoverBottles['Deployment'] = depNum
    if new_cnsm is None:
        new_cnsm = depdata.copy(deep=True)
        smoothed_cnsm = smoothed_data.copy(deep=True)
        deploy_bottles = deployBottles
        recover_bottles = recoverBottles
    else:
        new_cnsm = xr.concat([new_cnsm, depdata], dim='time')
        smoothed_cnsm = xr.concat([smoothed_cnsm , smoothed_data], dim='time')
        deploy_bottles = pd.concat([deploy_bottles, deployBottles])
        recover_bottles = pd.concat([recover_bottles, recoverBottles])

# Update the bottle corrected nitrate attributes
new_cnsm['bottle_corrected_nitrate'].attrs = {
    'long_name': 'Drift, Bottle, and T-S Corrected Dissolved Nitrate Concentration',
    'standard_name': 'mole_concentration_of_nitrate_in_sea_water',
    'units': 'umol L-1',
    'comment': ('The measured nitrate concentration that is corrected for temperature '
                'and salinity following Sakamoto (2009), '
                'for instrument drift using pre- and post-deployment calibration, and '
                'discrete water samples collected during deployment '
                'and recovery.')
}

# Update and apply the quality checks on the drift-corrected nitrate
qc_flags = quality_checks(new_cnsm, 'bottle_corrected_nitrate')
new_cnsm["bottle_corrected_nitrate_qc_flag"] = qc_flags
new_cnsm["bottle_corrected_nitrate_qc_flag"].attrs = {
    'flag_values': np.array([1, 2, 3, 4, 9]),
    'flag_meanings': 'pass not_evaluated suspect_or_of_high_interest fail missing_data',
    'standard_name': 'bottle_corrected_nitrate status_flag',
    'long_name': 'Nitrate Concentration - Temp, Sal, and Drift Corrected Quality Flag',
    'comment': ('This quality flag represents an assessment of the nitrate concentration '
                'that is corrected for temperature, salinity following Sakamoto (2009), '
                'for instrument drift, and discrete water samples collected during deployment '
                'and recovery. Checks include assessment of RMSE of the spectral '
                'measurements, absorptions at 254 nm and 350 nm wavelengths, dark values, '
                'spectral averages, and a range check based on instrument calibration.')
}

# Save the bottle values used for the corrections
cnsm_deploy_bottles = deploy_bottles
cnsm_recover_bottles = recover_bottles

# Save the results
cnsm.to_netcdf("/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/CNSM/CP01CNSM-RID26-07-NUTNRB000_bottle_corrected.nc", format='netcdf4', engine='h5netcdf')
smoothed_cnsm.to_netcdf("/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/CNSM/CP01CNSM-RID26-07-NUTNRB000_bottle_corrected_smoothed.nc", format='netcdf4', engine='h5netcdf')

##### CP04OSSM-RID26-07-NUTNRB000

In [ ]:
# Get the needed deployment information
site, node, sensor = 'CP04OSSM-RID26-07-NUTNRB000'.split('-',2)
deployInfo = get_deployment_info(site, node, sensor, deployments)
deployInfo = pd.DataFrame(deployInfo)
deployInfo.set_index(keys='deploymentNumber', drop=True, inplace=True)
deployInfo

In [ ]:
new_ossm = None
smoothed_ossm = None
deploy_bottles = None
recover_bottles = None
for depNum in np.unique(ossm['deployment']):
    depdata = ossm.where(ossm.deployment == depNum, drop=True)
    depdata, smoothed_data, deployBottles, recoverBottles = bottle_correction(depdata, deployInfo, ossm_nitrate)
    deployBottles['Deployment'] = depNum
    recoverBottles['Deployment'] = depNum
    if new_ossm is None:
        new_ossm = depdata.copy(deep=True)
        smoothed_ossm = smoothed_data.copy(deep=True)
        deploy_bottles = deployBottles
        recover_bottles = recoverBottles
    else:
        new_ossm = xr.concat([new_ossm, depdata], dim='time')
        smoothed_ossm = xr.concat([smoothed_ossm , smoothed_data], dim='time')
        deploy_bottles = pd.concat([deploy_bottles, deployBottles])
        recover_bottles = pd.concat([recover_bottles, recoverBottles])

# Update the bottle corrected nitrate attributes
new_ossm['bottle_corrected_nitrate'].attrs = {
    'long_name': 'Drift, Bottle, and T-S Corrected Dissolved Nitrate Concentration',
    'standard_name': 'mole_concentration_of_nitrate_in_sea_water',
    'units': 'umol L-1',
    'comment': ('The measured nitrate concentration that is corrected for temperature '
                'and salinity following Sakamoto (2009), '
                'for instrument drift using pre- and post-deployment calibration, and '
                'discrete water samples collected during deployment '
                'and recovery.')
}

# Update and apply the quality checks on the drift-corrected nitrate
qc_flags = quality_checks(new_ossm, 'bottle_corrected_nitrate')
new_ossm["bottle_corrected_nitrate_qc_flag"] = qc_flags
new_ossm["bottle_corrected_nitrate_qc_flag"].attrs = {
    'flag_values': np.array([1, 2, 3, 4, 9]),
    'flag_meanings': 'pass not_evaluated suspect_or_of_high_interest fail missing_data',
    'standard_name': 'bottle_corrected_nitrate status_flag',
    'long_name': 'Nitrate Concentration - Temp, Sal, and Drift Corrected Quality Flag',
    'comment': ('This quality flag represents an assessment of the nitrate concentration '
                'that is corrected for temperature, salinity following Sakamoto (2009), '
                'for instrument drift, and discrete water samples collected during deployment '
                'and recovery. Checks include assessment of RMSE of the spectral '
                'measurements, absorptions at 254 nm and 350 nm wavelengths, dark values, '
                'spectral averages, and a range check based on instrument calibration.')
}

# Save the bottle values used for the corrections
ossm_deploy_bottles = deploy_bottles
ossm_recover_bottles = recover_bottles

# Save the results
new_ossm.to_netcdf("/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/OSSM/CP04OSSM-RID26-07-NUTNRB000_bottle_corrected.nc", format='netcdf4', engine='h5netcdf')
smoothed_ossm.to_netcdf("/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/OSSM/CP04OSSM-RID26-07-NUTNRB000_bottle_corrected_smoothed.nc", format='netcdf4', engine='h5netcdf')

##### CP03ISSM-RID26-07-NUTNRB000

In [ ]:
# Get the needed deployment information
site, node, sensor = 'CP03ISSM-RID26-07-NUTNRB000'.split('-',2)
deployInfo = get_deployment_info(site, node, sensor, deployments)
deployInfo = pd.DataFrame(deployInfo)
deployInfo.set_index(keys='deploymentNumber', drop=True, inplace=True)
deployInfo

In [ ]:
new_issm = None
smoothed_issm = None
deploy_bottles = None
recover_bottles = None
for depNum in np.unique(issm['deployment']):
    depdata = issm.where(issm.deployment == depNum, drop=True)
    depdata, smoothed_data, deployBottles, recoverBottles = bottle_correction(depdata, deployInfo, issm_nitrate)
    deployBottles['Deployment'] = depNum
    recoverBottles['Deployment'] = depNum
    if new_issm is None:
        new_issm = depdata.copy(deep=True)
        smoothed_issm = smoothed_data.copy(deep=True)
        deploy_bottles = deployBottles
        recover_bottles = recoverBottles
    else:
        new_issm = xr.concat([new_issm, depdata], dim='time')
        smoothed_issm = xr.concat([smoothed_issm , smoothed_data], dim='time')
        deploy_bottles = pd.concat([deploy_bottles, deployBottles])
        recover_bottles = pd.concat([recover_bottles, recoverBottles])

# Update the bottle corrected nitrate attributes
new_issm['bottle_corrected_nitrate'].attrs = {
    'long_name': 'Drift, Bottle, and T-S Corrected Dissolved Nitrate Concentration',
    'standard_name': 'mole_concentration_of_nitrate_in_sea_water',
    'units': 'umol L-1',
    'comment': ('The measured nitrate concentration that is corrected for temperature '
                'and salinity following Sakamoto (2009), '
                'for instrument drift using pre- and post-deployment calibration, and '
                'discrete water samples collected during deployment '
                'and recovery.')
}

# Update and apply the quality checks on the drift-corrected nitrate
qc_flags = quality_checks(new_issm, 'bottle_corrected_nitrate')
new_issm["bottle_corrected_nitrate_qc_flag"] = qc_flags
new_issm["bottle_corrected_nitrate_qc_flag"].attrs = {
    'flag_values': np.array([1, 2, 3, 4, 9]),
    'flag_meanings': 'pass not_evaluated suspect_or_of_high_interest fail missing_data',
    'standard_name': 'bottle_corrected_nitrate status_flag',
    'long_name': 'Nitrate Concentration - Temp, Sal, and Drift Corrected Quality Flag',
    'comment': ('This quality flag represents an assessment of the nitrate concentration '
                'that is corrected for temperature, salinity following Sakamoto (2009), '
                'for instrument drift, and discrete water samples collected during deployment '
                'and recovery. Checks include assessment of RMSE of the spectral '
                'measurements, absorptions at 254 nm and 350 nm wavelengths, dark values, '
                'spectral averages, and a range check based on instrument calibration.')
}

# Save the bottle values used for the corrections
issm_deploy_bottles = deploy_bottles
issm_recover_bottles = recover_bottles

# Save the results
new_issm.to_netcdf("/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/ISSM/CP03ISSM-RID26-07-NUTNRB000_bottle_corrected.nc", format='netcdf4', engine='h5netcdf')
smoothed_issm.to_netcdf("/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/ISSM/CP03ISSM-RID26-07-NUTNRB000_bottle_corrected_smoothed.nc", format='netcdf4', engine='h5netcdf')

In [ ]:
# Merge all of the deployment and recovery bottles together and add their site
ossm_deploy_bottles['Site'] = 'OSSM'
ossm_recover_bottles['Site'] = 'OSSM'

issm_deploy_bottles['Site'] = 'ISSM'
issm_recover_bottles['Site'] = 'ISSM'

cnsm_deploy_bottles['Site'] = 'CNSM'
cnsm_recover_bottles['Site'] = 'CNSM'

deployBottles = pd.concat([cnsm_deploy_bottles, issm_deploy_bottles, ossm_deploy_bottles])
recoverBottles = pd.concat([cnsm_recover_bottles, issm_recover_bottles, ossm_recover_bottles])

In [ ]:
# Plot the individual deployment time series against the bottle observations
fig, ax = plt.subplots(figsize=(12,8))
site = 'CNSM'
depNum = 12
depdata = new_cnsm.where(new_cnsm.deployment == depNum, drop=True)
smoothed_data = smoothed_cnsm.where(smoothed_cnsm.deployment == depNum, drop=True)
dmask = (deployBottles['Site'] == site) & (deployBottles['Deployment'] == depNum) 
rmask = (recoverBottles['Site'] == site) & (recoverBottles['Deployment'] == depNum) 

a = '2015-11-01'
b = '2027-11-15'
# ax.plot(ctdbp.sel(time=slice(a,b))['time'], ctdbp.sel(time=slice(a,b))['sea_water_pressure'], marker=".", linestyle="", color="tab:blue")
ax.plot(depdata.sel(time=slice(a,b))['time'], depdata.sel(time=slice(a,b))['corrected_nitrate_concentration'], marker='.', linestyle="", color='tab:blue', label='T-S Corrected')
ax.plot(depdata.sel(time=slice(a,b))['time'], depdata.sel(time=slice(a,b))['drift_corrected_nitrate'], marker='.', linestyle="", color='tab:green', label='T-S, Drift Corrected')
ax.plot(depdata.sel(time=slice(a,b))['time'], depdata.sel(time=slice(a,b))['bottle_corrected_nitrate'], marker='.', linestyle="", color='tab:orange', label='T-S, Drift, Bottle Corrected')
#ax.plot(smoothed_data.sel(time=slice(a,b))['time'], smoothed_data.sel(time=slice(a,b))['drift_corrected_nitrate'], marker=".", linestyle="", color='black', label='6H Smoothed Data')
ax.plot(deployBottles[dmask].index, deployBottles[dmask]['Discrete Nitrate [uM]'], marker='o', linestyle='', color='tab:red', markeredgecolor='black', markersize=8, label='Discrete Bottle Sample')
ax.plot(recoverBottles[rmask].index, recoverBottles[rmask]['Discrete Nitrate [uM]'], marker='o', linestyle='', color='tab:red', markeredgecolor='black', markersize=8)
ax.grid()
ax.legend()
ax.set_ylabel('Nitrate Concentration [um/L]')
ax.set_title('Pioneer - New England Shelf Central Surface Mooring')
fig.autofmt_xdate()

In [ ]:
fig.savefig('../results/figures/newsletter_article_figure.png', facecolor='white', transparent=False, bbox_inches='tight', edgecolor='black')

Next, implement the quality checks on the bottle_corrected_nitrate

---
## Create the Final Datasets

Lastly, we want to take the datasets we've constructed above, clean them up, and save only the parameters of interest to us.

In [ ]:
# Identify the variables we are interested in 
save_vars = ['serial_number', 'deployment', 'sea_water_practical_salinity', 'sea_water_temperature', 'nitrate_concentration', 'nitrate_sensor_quality_flag',
             'corrected_nitrate_concentration', 'corrected_nitrate_concentration_qc_flag', 'drift_corrected_nitrate', 'drift_corrected_nitrate_qc_flag',
             'bottle_corrected_nitrate', 'bottle_corrected_nitrate_qc_flag', 'corrected_nitrate_concentration_mad']

In [ ]:
# Now do some cleanup - cut down to only the save parameters, rename a few variables for clearer name, update some attributes,
# and set the datatypes for a couple of the parameters that may have changed while manipulating data
# CNSM
cnsm_final = new_cnsm[save_vars]
cnsm_final = cnsm_final.rename({'corrected_nitrate_concentration_mad': 'burst_median_absolute_deviation', 
                   'nitrate_sensor_quality_flag': 'nitrate_concentration_qc_flag'})
cnsm_final['burst_median_absolute_deviation'].attrs['comment'] = ('The median absolute deviation calculated for each sampling burst.')
cnsm_final['drift_corrected_nitrate_qc_flag'] = cnsm_final['drift_corrected_nitrate_qc_flag'].astype('int')
cnsm_final['nitrate_concentration_qc_flag'] = cnsm_final['nitrate_concentration_qc_flag'].astype('int')
cnsm_final['deployment'] = cnsm_final['deployment'].astype('int')
cnsm_final['serial_number'] = cnsm_final['serial_number'].astype('int')

# ISSM
issm_final = new_issm[save_vars]
issm_final = issm_final.rename({'corrected_nitrate_concentration_mad': 'burst_median_absolute_deviation', 
                   'nitrate_sensor_quality_flag': 'nitrate_concentration_qc_flag'})
issm_final['burst_median_absolute_deviation'].attrs['comment'] = ('The median absolute deviation calculated for each sampling burst.')
issm_final['drift_corrected_nitrate_qc_flag'] = issm_final['drift_corrected_nitrate_qc_flag'].astype('int')
issm_final['nitrate_concentration_qc_flag'] = issm_final['nitrate_concentration_qc_flag'].astype('int')
issm_final['deployment'] = issm_final['deployment'].astype('int')
issm_final['serial_number'] = issm_final['serial_number'].astype('int')

# OSSM
ossm_final = new_ossm[save_vars]
ossm_final = ossm_final.rename({'corrected_nitrate_concentration_mad': 'burst_median_absolute_deviation', 
                   'nitrate_sensor_quality_flag': 'nitrate_concentration_qc_flag'})
ossm_final['burst_median_absolute_deviation'].attrs['comment'] = ('The median absolute deviation calculated for each sampling burst.')
ossm_final['drift_corrected_nitrate_qc_flag'] = ossm_final['drift_corrected_nitrate_qc_flag'].astype('int')
ossm_final['nitrate_concentration_qc_flag'] = ossm_final['nitrate_concentration_qc_flag'].astype('int')
ossm_final['deployment'] = ossm_final['deployment'].astype('int')
ossm_final['serial_number'] = ossm_final['serial_number'].astype('int')

In [ ]:
# Save the final versions
cnsm_final.to_netcdf('/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/CNSM/CP01CNSM-RID26-07-NUTNRB000_final.nc', format='netcdf4', engine='h5netcdf')

issm_final.to_netcdf('/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/ISSM/CP03ISSM-RID26-07-NUTNRB000_final.nc', format='netcdf4', engine='h5netcdf')

ossm_final.to_netcdf('/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/OSSM/CP04OSSM-RID26-07-NUTNRB000_final.nc', format='netcdf4', engine='h5netcdf')

In [ ]:
# Save the deployment and recovery bottle datasets
deployBottles['Type'] = 'Deployment'
recoverBottles['Type'] = 'Recovery'
all_bottles = pd.concat([deployBottles, recoverBottles])
all_bottles.to_csv('/home/jovyan/Curated_datasets/nitrate_validation/data/Pioneer-NES/Ship/Bottle_Nitrate.csv')